# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [56]:
# Dependencies and Setup
import matplotlib.pyplot as plt
from citipy import citipy
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
city_df = pd.read_csv('city_weather_data.csv')

city_df.head()

,city Name,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lng,Latitude,Date
0,Punta Arenas,50.00,87,20,8.05,-70.92,-53.15,3/4/2020
1,Busselton,88.79,10,59,9.08,115.33,-33.65,3/4/2020
2,Taoudenni,67.60,13,0,6.51,-3.98,22.68,3/4/2020
3,Roald,36.18,56,100,11.41,6.13,62.58,3/4/2020
4,Jaru,71.49,99,99,2.15,-62.47,-10.44,3/4/2020


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:

locations = city_df[["Latitude", "Lng"]]
gmaps.configure(api_key=g_key)
gmap_weight =city_df["Humidity (%)"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=gmap_weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
df_cities=pd.DataFrame(city_df)
filtered_df = df_cities.loc[((df_cities["Temperature (F)"]>70.0) & (df_cities["Temperature (F)"]<80.0))
                   &(df_cities["Wind Speed (mph)"]<10.0)&(df_cities["Cloudiness (%)"]==0),:]

filtered_df.dropna()
print(len(filtered_df))


17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df =pd.DataFrame(filtered_df)
hotel_df["Hotel Name"] = ""
hotel_df.head()


,city Name,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lng,Latitude,Date,Hotel Name
68,Port Alfred,71.01,94,0,7.81,26.89,-33.59,3/4/2020,
77,Nioro,72.00,12,0,3.89,-15.75,13.35,3/4/2020,
119,Puerto Ayacucho,73.76,75,0,3.02,-67.62,5.66,3/4/2020,
136,Bhuj,79.03,46,0,8.32,69.67,23.27,3/4/2020,
161,San Miguel de Tucumán,73.40,78,0,3.36,-65.22,-26.82,3/4/2020,


In [48]:

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key}

for index, row in hotel_df.iterrows():
  
    lat = row["Latitude"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"

    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    #url = "https://maps.googleapis.com/maps/api/place/nearbysearch/output?parameters"

    hotel_search = requests.get(url, params=params)
    
    hotel_search = hotel_search.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_search["results"][0]["name"]
    except:
        hotel_df.loc[index, "Hotel Name"] = (np.nan)

hotel_df.head(60)

,city Name,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Lng,Latitude,Date,Hotel Name
68,Port Alfred,71.01,94,0,7.81,26.89,-33.59,3/4/2020,Halyards
77,Nioro,72.00,12,0,3.89,-15.75,13.35,3/4/2020,NaN
119,Puerto Ayacucho,73.76,75,0,3.02,-67.62,5.66,3/4/2020,Hotel Apure
136,Bhuj,79.03,46,0,8.32,69.67,23.27,3/4/2020,City The Village Resort
161,San Miguel de Tucumán,73.40,78,0,3.36,-65.22,-26.82,3/4/2020,Catalinas Park
215,Chapada dos Guimarães,75.20,78,0,2.24,-55.75,-15.46,3/4/2020,Pousada Penhasco
271,Simao,73.74,29,0,7.92,100.97,22.79,3/4/2020,Pu'er Meizihu New Century Hot Spring Hotel
273,Bubaque,79.99,65,0,9.55,-15.83,11.28,3/4/2020,Ponta Anchaca
318,Mercedes,71.98,87,0,6.51,-59.43,-34.65,3/4/2020,Gran Hotel Mercedes
326,Bodden Town,78.37,80,0,3.00,-81.25,19.28,3/4/2020,Turtle Nest Inn


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = '''
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city Name}</dd>

</dl>
'''
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Lng"]]

In [55]:
# Add marker layer ontop of heat map
gmarkers = gmaps.marker_layer(locations)
fig.add_layer(gmarkers)
fig


# Display Map

Figure(layout=FigureLayout(height='420px'))